# Выделение факторов и предсказание
### Инструкция
Соберите случайный ансамбль из нескольких методов выделения факторов - корреляции, взаимной информации, важности признаков, главных компонент, независимых компонент и др. Получите не менее 3 наборов из 5 наиболее важных признаков.

Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.

Используя эти ансамбли, предскажите продолжительность жизни на 2019 год.

Данные:
https://video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

### Общий ход решения
1. подготовка данных
2. выделение значимых факторов:
    - выделяются три набора значимых факторов: два методом случайного просеивания, 1 - по взаимной корреляции факторов  
3. Для каждого из наборов выполняется прогноз с помощью ансамбля различных моделей
4. Окончательный результат получается усреднением

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import f_regression,mutual_info_regression
from catboost import CatBoostRegressor
from sklearn.decomposition import PCA,FastICA,TruncatedSVD,NMF


In [2]:
# (экстраполяция на краях) Находим лин.регрессию для прогноза значений 

def linear_extrapolation (x):
    X = np.array(x.dropna().index.astype(int)).reshape(-1, 1)
    Y = np.array(x.dropna().values).reshape(-1, 1)
    if X.shape[0] > 0:
        f = LinearRegression().fit(X, Y)
        for i in x.index:
            v = x.loc[i]
            if v != v:
                # если значение nan, вычисляем прогнозное и берем его 
                v = f.predict([[int(i)]])[0][0]
                
                # если прогноз дал значение меньше нуля - заменяем на ноль
                if v < 0:
                    v = 0
                    
                x.loc[i] = v
    return x

## Загрузка и очистка данных
Линейно интерполируем пропуски, а начальные и конечные пропущенные данные экстраполируем по линейному закону. Удаляем пустые параметры

In [3]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv",
                   na_values=["-", " - ","...","…"," -"])

In [4]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

data = data.apply(linear_extrapolation, axis=1, result_type="expand")

data["feature"] = features
data.dropna(inplace=True)
features = np.array(data["feature"])

data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = features
data.head(10)

,ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.1. ЧИСЛО ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ (на конец года),ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.2. ОБОРОТ ОРГАНИЗАЦИЙ (миллиардов рублей),12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.1. Число малых предприятий,12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.2. Число малых предприятий на 10 000 человек населения,12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.3. Среднесписочная численность работников (без внешних совместителей),12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.4. Оборот малых предприятий,"12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.1. Число малых предприятий в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.2. Число малых предприятий на 10 000 человек населения в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.3. Среднесписочная численность работников (без внешних совместителей) в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.4. Выручка от реализации товаров (работ, услуг) малых предприятий в 2010, 2015 гг.",...,4.22. ЧИСЛО ФИЛИАЛОВ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ ВЫСШЕГО ОБРАЗОВАНИЯ (на начало учебного года),"4.23. ЧИСЛЕННОСТЬ ПРОФЕССОРСКО-ПРЕПОДАВАТЕЛЬСКОГО ПЕРСОНАЛА , ОСУЩЕСТВЛЯЮЩЕГО ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ПРОГРАММАМ ВЫСШЕГО ОБРАЗОВАНИЯ (на начало учебного года; человек)","4.24. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ (на начало учебного года; тысяч человек)","4.25. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ на 10 000 человек населения (на начало учебного года; человек)","4.26. ПРИЕМ НА ОБУЧЕНИЕ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ (тысяч человек)","4.27. ВЫПУСК БАКАЛАВРОВ, СПЕЦИАЛИСТОВ, МАГИСТРОВ (тысяч человек)","4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.1. Число организаций (на конец года)","4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.2. Численность аспирантов (на конец года; человек)","4.29. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ ДОКТОРАНТОВ 4.29.1. Число организаций (на конец года)","4.29. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ ДОКТОРАНТОВ 4.29.2. Численность докторантов (на конец года; человек)"
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.5,0.453901
1,0.085716,0.000000,0.008395,0.043234,0.052010,0.000000,0.058462,0.058462,0.058462,0.000000,...,0.950834,0.945835,0.228175,0.225926,0.50625,0.093248,0.166667,0.135710,0.6,0.453901
2,0.160766,0.000000,0.065825,0.098528,0.104020,0.000000,0.116923,0.116923,0.116923,0.000000,...,0.901667,0.891671,0.432540,0.429630,0.50000,0.176849,0.583333,0.269543,0.7,0.489362
3,0.263245,0.023087,0.123254,0.153821,0.156030,0.000000,0.175385,0.175385,0.175385,0.000000,...,0.852501,0.837506,0.638889,0.637037,0.77500,0.292605,0.666667,0.402126,0.6,0.574468
4,0.342423,0.083656,0.180684,0.209115,0.208040,0.000000,0.233846,0.233846,0.233846,0.070488,...,0.850417,0.783342,0.792659,0.792593,0.85000,0.440514,0.833333,0.411507,0.6,0.531915
5,0.450100,0.171797,0.238114,0.264409,0.260049,0.000000,0.292308,0.292308,0.292308,0.141415,...,0.583619,0.729177,0.941468,0.940741,0.95625,0.540193,0.833333,0.376485,0.5,0.496454
6,0.449605,0.208053,0.295543,0.319702,0.312059,0.006489,0.350769,0.350769,0.350769,0.212342,...,0.566945,0.675013,1.000000,1.000000,1.00000,0.710611,1.000000,0.435897,0.5,0.560284
7,0.497923,0.277258,0.352973,0.374996,0.364069,0.082067,0.409231,0.409231,0.409231,0.283268,...,0.583619,0.620848,0.998016,0.996296,0.91875,0.836013,1.000000,0.524078,0.5,0.602837
8,0.596958,0.341322,0.410403,0.430290,0.416079,0.157646,0.467692,0.467692,0.467692,0.354195,...,0.566945,0.566683,0.957341,0.955556,0.63125,0.826367,1.000000,0.668543,0.6,0.609929
9,0.630685,0.342645,0.467832,0.485583,0.468089,0.233224,0.526154,0.526154,0.526154,0.425121,...,0.566945,0.512519,0.922619,0.914815,0.48750,0.900322,0.916667,0.752345,0.7,0.666667


In [5]:
num_factors = 10

y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

## Выделение значащих факторов
Сузим данные до значащих факторов

In [6]:
# Ансамбль матричных методов выделения факторов

def fit_mtx_ensemble(x,y):
    pca5 = PCA(n_components=5, random_state=42).fit(x)
    pca10 = PCA(n_components=10, random_state=42).fit(x)
    pca18 = PCA(n_components=12, random_state=42).fit(x)
    svd5 = TruncatedSVD(n_components=5, random_state=42, n_iter=100).fit(x)
    svd10 = TruncatedSVD(n_components=10, random_state=42, n_iter=100).fit(x)
    svd18 = TruncatedSVD(n_components=12, random_state=42, n_iter=100).fit(x)
    nmf5 = NMF(n_components=5, random_state=42, max_iter=5000).fit(x)
#     ica5 = FastICA(n_components=5, random_state=42, whiten=False).fit(x)
#     ica10 = FastICA(n_components=10, random_state=42, whiten=False).fit(x)
#     ica18 = FastICA(n_components=18, random_state=42, whiten=False).fit(x)
#     nmf10 = NMF(n_components=10, random_state=42, max_iter=1000).fit(x)
#     nmf18 = NMF(n_components=18, random_state=42, max_iter=5000).fit(x)
    
    models = [pca5, pca10, pca18, svd5, svd10, svd18,  nmf5] #, nmf10, nmf18], ica5, ica10, ica18,
    mdl_sum = 0
    for i,m in enumerate(models):
        a = MinMaxScaler().fit_transform(np.array(m.components_).T)
        #+отладка print(a.T.shape)  
        for c in a.T:
            mdl_sum+= np.abs(c)
    return {     
        "mtx_e": np.argsort(np.abs(mdl_sum)[::-1])
    }

In [7]:
# Ансамбль из методов взаимной информации, важности признаков и др

def fit_info_ensemble(x, y):
    mi = mutual_info_regression(x, y, random_state=17, n_neighbors=3)
    mi /= np.max(mi)   # нормирование

    return {
        # словарь из наборов индексов, отсортированных по убыванию коэф.         
        "mi":     np.argsort(np.abs(mi))[::-1],

        "rfr42":   np.argsort(np.abs(RandomForestRegressor(n_estimators=250, random_state=42).fit(x, y).feature_importances_))[::-1],
        "etr42":   np.argsort(np.abs(ExtraTreesRegressor(n_estimators=250, random_state=42).fit(x, y).feature_importances_))[::-1]
    }
        
    
# ensemble = rfr.feature_importances_ + etr.feature_importances_ + mi + l2 + l1

In [8]:
# Ансамбль взаимной корреляции признаков

def inter_korr(alpha=0.05):
    l2, l1 = [], []
    for column in data.columns:
        if column != y_column and column not in y_columns:
            x = np.array(data[column]).reshape(-1, 1)
            y = data[y_column]
            l1.append(Lasso(alpha=0.01).fit(x, y).score(x, y))
            l2.append(Ridge(alpha=alpha).fit(x, y).score(x, y))
    return(np.array(l1) + np.array(l2))


In [9]:
kor_ensemble = inter_korr()
# print(kor_ensemble.shape)

kor_factors = np.argsort(kor_ensemble)[::-1][:10]
features[kor_factors]
print(kor_factors)

[364 273  21 188 330 139 201 114 250 249]


In [12]:
def random_super_features(fit_ensemble):
    super_features = np.zeros(len(features))
    subset_size = 50
    subset_power = 20
    subset_iter = 50
    subset_super = 20
    for k in range(subset_super):
        print(f"<super={k}")
        subset_features = np.zeros(len(features))
        for j in range(subset_iter):
            features_rank = [0]*len(features)
            if j % 10 == 0:
                print(f"  iter={j}") 

            for f_random in np.random.randint(len(features), size=[subset_power, subset_size]):
                x_subset = data[features[f_random]].copy()  # Возобновление исходных данных
                # Проверка на наличие в случайном наборе целевых (основной и дублирующих)
                if y_column in features[f_random]:
                    x_subset = x_subset.drop(labels=[y_column], axis=1)
                # Убрать дублирующие                
                for y_column_ in y_columns:
                    if y_column_ in features[f_random]:
                        x_subset = x_subset.drop(labels=[y_column_], axis=1)

                # обучение всех моделей в ансамбле
                ensemble = fit_ensemble(x_subset, y)

                # для каждой обученной модели            
                for e in ensemble:
                    i = 0
                    for f in ensemble[e]:
                        # увеличение суммарного ранга в зависимости от позиции 0=50, 1=49,..50=0                   
                        features_rank[f_random[f]] += subset_size - i
                        i += 1

            subset_features[np.argsort(features_rank)[::-1][:10]] += 1
        super_features[np.argsort(subset_features)[::-1][:10]] += 1
    print(super_features.shape)    
    return np.argsort(super_features)[::-1][:10]

In [11]:
matrix_random_set1 = random_super_features(fit_mtx_ensemble)
print(matrix_random_set1)
print(features[matrix_random_set1])


<super=0
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=1
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=2
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=3
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=4
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=5
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=6
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=7
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=8
  iter=0
  iter=5
  iter=10
  iter=15
  iter=20
  iter=25
  iter=30
  iter=35
  iter=40
  iter=45
<super=9
  iter=0
  iter=5
  iter=10


In [13]:
info_random_set1 = random_super_features(fit_info_ensemble)
print(info_random_set1)
print(features[info_random_set1])

<super=0
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=1
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=2
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=3
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=4
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=5
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=6
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=7
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=8
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=9
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=10
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=11
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=12
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=13
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=14
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=15
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=16
  iter=0
  iter=10
  iter=20
  iter=30
  iter=40
<super=

## Ансамбль предсказаний

**Задание** 
Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.

In [14]:
def fit_predict_ensemble(sel_features, retries=50 ):
    y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
    y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
                "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
    y = data[y_column]
    x = data[[features[i] for i in sel_features]]
    
    # Сдвинем факторы на год назад: чтобы предсказывать по значениям текущего года - следующий
    x_ = x[:-1]
    y_ = np.array(y[1:])
    
    # x_.head()
    # print(x_.shape, y_.shape)

    rf_fit = []
    for i in range (retries):
        rf_fit.append(RandomForestRegressor(n_estimators=50, random_state=i).fit(x_, y_))

    et_fit = []
    for i in range (retries):
        et_fit.append(ExtraTreesRegressor(n_estimators=50, random_state=i).fit(x_, y_))

    from catboost import CatBoostRegressor
    cb_fit = []
    for i in range (retries):
        cb_fit.append(CatBoostRegressor(iterations=20, depth=16, random_state=i, verbose=False).fit(x_, y_))

    lasso_fit = []
    for i in range (retries):
        lasso_fit.append(Lasso(alpha=0.1,random_state=i).fit(x_, y_))

    enet_fit = []
    for i in range (retries):
        enet_fit.append(ElasticNet(alpha=0.8,random_state=i).fit(x_, y_))
    
    # pr = []
    # for i in range (RETRIES):
    #     pr.append(rf_fit[i].predict(x[-1:]))
    #     pr.append(et_fit[i].predict(x[-1:]))
    #     pr.append(cb_fit[i].predict(x[-1:]))
    #     pr.append(lasso_fit[i].predict(x[-1:]))
    #     pr.append(enet_fit[i].predict(x[-1:]))
    # npr = np.array(pr)
    # prediction = npr.mean()
    # print(npr.shape,prediction, npr.std())
    
    # С меньшими затратами памяти
    prediction = 0
    for i in range (retries):
        prediction += rf_fit[i].predict(x[-1:])
        prediction += et_fit[i].predict(x[-1:])
        prediction += cb_fit[i].predict(x[-1:])
        prediction += lasso_fit[i].predict(x[-1:])
        prediction += enet_fit[i].predict(x[-1:])
    prediction /= (retries*5)
    
    return prediction
    

## Формирование предсказаний 

In [15]:
def life_expectancy(factors, steps=100):
    prediction = fit_predict_ensemble(factors, retries=steps)
#     print(prediction) 
    # возвращаем начальные значения без нормализации
    data_predict = data.copy()
    data_predict[y_column] = np.ones(len(data_predict)) * prediction
    # data_predict[y_column].head()
    data_predict = scaler.inverse_transform(data_predict)
    
    le = round(data_predict[0][np.where(features==y_column)][0],2)
    # print(f"Прогноз продолжительности жизни = {le} г")
    
    return le
    

In [19]:
factors_list = [kor_factors, info_random_set1, matrix_random_set1]
result = 0
for factors in factors_list:
    le = life_expectancy(factors,50)
    result += le
    print(le)

result /= len(factors_list)

print(f"Прогноз продолжительности жизни = {result:.2f} г")

72.85
72.84
72.86
Прогноз продолжительности жизни = 72.85 г
